|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 6:</h2>|<h1>Intervention (causal) mech interp<h1>|
|<h2>Section:</h2>|<h1>Editing hidden states<h1>|
|<h2>Lecture:</h2>|<h1><b>CodeChallenge HELPER: Measure and correct BERT's bias<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F

import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# Exercise 1: Hook the BERT model

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM

# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertForMaskedLM.from_pretrained('bert-large-uncased')

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# push the model to the GPU and switch to eval mode

In [ ]:
# indices (redefined later)
layer2replace = 40000 # no replacement...
hs_vector2replace = torch.zeros(


mixture = [.1,.9]

# hooking functions
def implant_hook(layer_number):
  def hook(module, input, output):

    # only change this layer if there's a matching variable value
    if layer_number == layer2replace:

      # unpack tuple
      hidden,

      # mix the old and the new
      hidden[0,maskTarget_idx,:] =  +


      # reconstruct output
      output =

      print(f'Replaced layer {layer_number:2}')

    return output
  return hook


# loop over layers and do surgery
handles = []
for layeri in range(model.config.num_hidden_layers):
  h = model.bert.encoder.layer[layeri].
  handles.append

# Exercise 2: Test for a gender bias in BERT

In [ ]:
# list of target words
target_words = [ 'he','she','they' ]

# tokenize sentences
tokens_he   = tokenizer(f'The engineer informed the client that he would need more time.',return_tensors='pt')
tokens_she  =
tokens_they =

# tokenize the masked sentence
tokens_mask =

In [ ]:
# the mask index
maskTarget_idx =

# token indices of target words
targets_idx =

# print out the tokens
for t in tokens_mask['input_ids'][0]:
  print(f

print(f'\nThe mask is in token index {maskTarget_idx}\n')
for t in targets_idx:
  print(f

In [ ]:
# redefine as outside the range, in case you want to rerun this code later
layer2replace = 40000

# forward-pass the four versions
with torch.no_grad():
  out_he = model(**tokens_he,)
  out_she =
  out_they =
  out_mask =

In [ ]:
# grab and visualize the log-softmax

fig,axs = plt.subplots(2,3,figsize=(10,5))

# for "he"
logsm = F.log_softmax(out_he.logits
axs[0,0].bar(range(3),logsm[])
axs[1,0].bar(range(3),torch.exp(logsm[]))
axs[0,0].set(xticks=range(3),xticklabels=target_words,ylabel='Log-softmax',title='Probs. in "he" sentence')
axs[1,0].set(xticks=range(3),xticklabels=target_words,xlabel='Target words',ylabel='Softmax prob')


# for "she"


# for "they"



plt.tight_layout()
plt.show()

In [ ]:
# grab and visualize the log-softmax
logsm = F.log_softmax(out_mask.l

fig,axs = plt.subplots(1,2,figsize=(10,3.5))

axs[0].bar(range(3),
axs[1].bar(range(3),



fig.suptitle(tokenizer.decode(tokens_mask['input_ids'][0,1:-1]),fontweight='bold')

plt.tight_layout()
plt.show()

# Exercise 3: Edit in an anti-bias?

In [ ]:
# get s/he/they activation from one hidden state

layer2replace =
hs_vector2replace =

with torch.no_grad():
  out_mask_replace = model(**tokens_mask.to(device),output_hidden_states=True)

In [ ]:
# grab and visualize the log-softmax
logsm_orig =
logsm_repl =

fig,axs = plt.subplots(1,2,figsize=(10,3.5))

axs[0].bar(label='Original')
axs[0].bar(,label='Modified')
axs[0].legend()


axs[1].

fig.suptitle(tokenizer.decode(tokens_mask['input_ids'][0,1:-1]),fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
bias_orig =
bias_repl =

print(f'Bias (he-she) in original model: {bias_orig:.3f}')
print(f'Bias (he-she) in modified model: {bias_repl:.3f}')

# Exercise 4: Laminar profile of anti-bias impact

In [ ]:
mixture = [.5,.5]

In [ ]:
bias_scores = torch.zeros(model.config.num_hidden_layers)

for layer2replace in range(

  # vector to replace (from "she" sentence)
  hs_vector2replace = out_she.hidden_stat

  # forward-pass with hook to replace
  with torch.no_grad():
    out_mask_replace = model(,=True)

  # calculate the log-sm probabilities
  logsm_repl =

  # calculate the bias towards "he"
  bias_scores[layer2replace] =

In [ ]:
plt.figure(figsize=(8,3))
plt.plot(bias_scores,'ko-',markerfacecolor=[.7,.9,.7],markersize=10,linewidth=.5)


In [ ]:
for h in handles:
  h.remove()